#Create customized dataset for few shot video2video

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### Check CUDA devices exist
import torch
print("Is CUDA available", torch.cuda.is_available())
print("Cuda device count is ",torch.cuda.device_count())
print("Current CUDA device is", torch.cuda.current_device())

Is CUDA available True
Cuda device count is  1
Current CUDA device is 0


## Download Files

In [ ]:
!pip install --upgrade youtube-dl
from __future__ import unicode_literals
import youtube_dl
### Download Files
import os
import requests
from urllib import request

### Select driver video sequence
vid_url = 'https://www.dropbox.com/s/fb4wkjq220xv9ex/driver.mp4?dl=1'
vid_url = 'https://www.dropbox.com/s/2w7oiifjrfqtrji/avatar.mp4?dl=1'

# change url to dl=1 


### Select reference image
#ref_url = 'https://www.dropbox.com/s/ga8jcxhpglnyv0o/woman_1.jpg?dl=1'
#ref_url = 'https://www.dropbox.com/s/ul63i5plsrjtuoq/avatar_1.jpg?dl=1'
ref_url = 'https://i.guim.co.uk/img/media/cabcfbdafc3b7e0854e07c35c35b18c199aa19a2/0_0_5130_3078/master/5130.jpg?width=1200&height=1200&quality=85&auto=format&fit=crop&s=e954cf8284c34c478581b7bae10d694e'

In [ ]:
### Download youtube video
youtube = False # change to True to use a Youtube link as driver video
youtube_vid_url = 'https://www.youtube.com/watch?v=y6pVw4gjH2M&ab_channel=F1Racing'

In [ ]:

### Download dataset locally
sample_path = '/content/demo_dataset/'
if not os.path.exists(sample_path):
    os.makedirs(sample_path)
ref_save_path =  '/content/demo_dataset/ref.jpg'
vid_save_path = '/content/demo_dataset/driver.mp4'
request.urlretrieve(ref_url, ref_save_path) # save pic locally
if youtube:
  ydl_opts  = {
    'outtmpl': '/content/demo_dataset/driver.mp4', 
      } 
  with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    info = ydl.download([youtube_vid_url])
else:
  request.urlretrieve(vid_url, vid_save_path)


### Define paths
PATH_TO_DATA = '/content/demo_dataset/'
if not os.path.exists(PATH_TO_DATA):
    os.mkdir(PATH_TO_DATA)
ref_img = str(ref_save_path) # define local path to ref image
driver_vid = str(vid_save_path) # define local path to driver video sequence


### Define DensePose & OpenPose paths
openpose_path = '/content/drive/MyDrive/openpose/'
densepose_path = '/content/drive/MyDrive/DensePose/'
output_path = '/content/drive/MyDrive/test'

## Download Openpose

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  
from IPython.display import YouTubeVideo

Selecting previously unselected package libgflags2.2.
(Reading database ... 155455 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

## Customized Test Dataset Structure
Check the following structure for your customized dataset

In [ ]:
### Customized Test Dataset
#
#                                          |---------landmarks-dlib68
#                       |---------- driving|
#                       |                  |---------images
# face_forensics -------|
#                       |                    |-----landmarks-dlib68
#                       |---------- reference|
#                                            |-----images

In [ ]:
#!rm -r /content/output/ # remove directory
#!rm -r /content/render_dir/ 
!mkdir /content/output/ # make directory
!mkdir /content/output/driving/
!mkdir /content/output/reference/
!mkdir /content/render_dir/ #make a directory to save rendered images with keypoints

## Run Openpose

For more information, check the openpose flags [here](https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/include/openpose/flags.hpp)

In [ ]:
### Test that videos and images are in correct format
%cd /content/openpose
!./build/examples/openpose/openpose.bin --display 0  \
          --disable_blending --render_pose 2 --video /content/demo_dataset/driver.mp4 \
          --write_images /content/sample_data/ --face --hand  \
          --face_render_threshold 0.1 --hand_render_threshold 0.02 

/content/openpose
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 39.867000 seconds.


### Locally

In [ ]:
import os
### Run Openpose on video (keypoint extraction)
openpose_root = '/content/openpose/'
video_path = '/content/demo_dataset/driver.mp4'
render_dir = '/content/render_dir/driving/'
op_dir = '/content/output/driving/landmarks-dlib68/'
frames_dir = '/content/output/driving/images/'
root = '../../'
cmd = 'cd %s; ./build/examples/openpose/openpose.bin --display 0 ' \
          '--disable_blending --render_pose 2 --video %s ' \
          ' --write_images %s --face --hand ' \
          '--face_render_threshold 0.1 --hand_render_threshold 0.02 ' \
          '--write_json %s; cd %s' \
          %(openpose_root, video_path, render_dir, op_dir, root)
os.system(cmd)

### Extract also keyframes
#!cd openpose && ./build/examples/openpose/openpose.bin --video ../demo_dataset/driver.mp4 --write_json ../drive/MyDrive/openpose_output/ --render_pose 0 --display 0  --write_images ../face_forensics/ --write_video ../openpose.avi
cmd = 'cd %s; ./build/examples/openpose/openpose.bin --display 0 ' \
          '--disable_blending --render_pose 0 --video %s ' \
          '--write_images %s --write_images_format "jpg" --face --hand ' \
          '--face_render_threshold 0.1 --hand_render_threshold 0.02 ' \
          '--write_json %s; cd %s' \
          %(openpose_root, video_path, frames_dir, op_dir, root)
os.system(cmd)

0

In [ ]:
### Run Openpose on image
openpose_root = '/content/openpose/'
img_dir = '/content/demo_dataset/'
render_dir = '/content/render_dir/reference'
op_dir = '/content/output/reference/landmarks-dlib68/'
frames_dir = '/content/output/reference/images/'
root = '../../'
cmd = 'cd %s; ./build/examples/openpose/openpose.bin --display 0 ' \
          '--disable_blending --render_pose 2 --image_dir %s ' \
          '--write_images %s --face --hand ' \
          '--face_render_threshold 0.1 --hand_render_threshold 0.02 ' \
          '--write_json %s; cd %s' \
          % (openpose_root, img_dir, render_dir, op_dir, root)
os.system(cmd)
#!cd openpose && ./build/examples/openpose/openpose.bin --image_dir ../drive/MyDrive/test_data/faceForensics/driving/images --render_pose 2 --write_json ../output/reference --display 0   --write_images ../ref_openpose

### Extract also keyframes
#!cd openpose && ./build/examples/openpose/openpose.bin --video ../demo_dataset/driver.mp4 --write_json ../drive/MyDrive/openpose_output/ --render_pose 0 --display 0  --write_images ../face_forensics/ --write_video ../openpose.avi
cmd = 'cd %s; ./build/examples/openpose/openpose.bin --display 0 ' \
          '--disable_blending --render_pose 0 --image_dir %s ' \
          '--write_images %s --write_images_format "jpg" --face --hand ' \
          '--face_render_threshold 0.1 --hand_render_threshold 0.02 ' \
          '--write_json %s; cd %s' \
          % (openpose_root, img_dir, frames_dir, op_dir, root)
os.system(cmd)

0

## Finalize custom dataset

### Convert json to dlib-68 landmarks
Openpos JSON output has 2 more landmarks for the 2 pupils of the face (70 keypoints/landmarks). Try removing them, to convert to the appropriate dlib-68 
format

In [ ]:
def convert_openpose_to_dlib68(filename):
  'Convert openpose json landmark format to dlib68 format and overwrite'
  import json
  import numpy as np
  # JSON file
  #filename = '/content/output/reference/landmarks-dlib68/ref_keypoints.json'
  f = open (filename, "r")

  # Reading from json file
  data = json.loads(f.read())

  # Convert to lib68, by rearranging to 3D array and then ignore confidence
  landmarks = data['people'][0]['face_keypoints_2d'] # x,y position of landmarks
  landmarks = np.array(landmarks) # convert to numpy array
  dlib68 = np.round((np.reshape(landmarks, (70, 3)))).astype(int)[0:68, 0:2]

  # Save new json landmark file
  dlib68_json =  json.dumps(dlib68.tolist()) # convert to JSON array 
  with open(filename, 'w') as outfile:
    outfile.write(dlib68_json)

  return 'Conversion complete'

In [ ]:
dataset_root = '/content/output'
for path, subdirs, files in os.walk(dataset_root):
  for name in files:
    if '.json' in name:
      fullpath  = os.path.join(path, name)
      print(fullpath)
      try:
        convert_openpose_to_dlib68(fullpath)
      except:
        print("File %s could not convert"%fullpath)
print("Conversion complete")
    

/content/output/reference/landmarks-dlib68/ref_keypoints.json
/content/output/driving/landmarks-dlib68/driver_000000000186_keypoints.json
/content/output/driving/landmarks-dlib68/driver_000000000179_keypoints.json
/content/output/driving/landmarks-dlib68/driver_000000000101_keypoints.json
/content/output/driving/landmarks-dlib68/driver_000000000038_keypoints.json
/content/output/driving/landmarks-dlib68/driver_000000000117_keypoints.json
/content/output/driving/landmarks-dlib68/driver_000000000111_keypoints.json
/content/output/driving/landmarks-dlib68/driver_000000000012_keypoints.json
/content/output/driving/landmarks-dlib68/driver_000000000010_keypoints.json
/content/output/driving/landmarks-dlib68/driver_000000000104_keypoints.json
/content/output/driving/landmarks-dlib68/driver_000000000109_keypoints.json
/content/output/driving/landmarks-dlib68/driver_000000000098_keypoints.json
/content/output/driving/landmarks-dlib68/driver_000000000226_keypoints.json
/content/output/driving/la

### Rename files
Rename files, to  match the appropriate dataset format for the **fs_vid2vid**

In [ ]:
### Define dataset root
dataset_root = '/content/output'
for path, subdirs, files in os.walk(dataset_root):
  counter = 0
  files.sort()
  for name in files:
    oldpath  = os.path.join(path, name)
    new_name = str(counter).zfill(5) + '.'+name.split('.')[-1]
    newpath = os.path.join(path, new_name)
    os.rename(oldpath, newpath)# rename 
    counter = counter+1

### Download locally

In [ ]:
!zip -r /content/faceForensics.zip  /content/output
from google.colab import files
files.download("/content/faceForensics.zip")

  adding: content/output/ (stored 0%)
  adding: content/output/reference/ (stored 0%)
  adding: content/output/reference/images/ (stored 0%)
  adding: content/output/reference/images/00000.jpg (deflated 6%)
  adding: content/output/reference/landmarks-dlib68/ (stored 0%)
  adding: content/output/reference/landmarks-dlib68/00000.json (deflated 62%)
  adding: content/output/driving/ (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Upload dataset to Google Drive
Rename the dataset to *faceForensics* and upload it to GoogleDrive. Now it can be used by the inference notebook, to test **fs_vid2vid** to your customized dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!rm -r /content/drive/MyDrive/faceForensics/
!mv /content/output/ /content/drive/MyDrive/faceForensics #move to another directory

## Visualize results  & Play Around

In [ ]:
### Run Openpose
# remove render pose to enable pose estimation
!cd openpose && ./build/examples/openpose/openpose.bin --video ../demo_dataset/driver.mp4 --write_json ../drive/MyDrive/openpose_output/ --render_pose 0 --display 0  --write_images ../face_forensics/ --write_video ../openpose.avi
# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi o.mp4

/bin/bash: line 0: cd: openpose: No such file or directory
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --en

In [ ]:
!cd /content/openpose/
!./build/examples/openpose/openpose.bin --video ../demo_dataset/driver.mp4 --render_pose 2  --face --hand \
          --face_render_threshold 0.1 --hand_render_threshold 0.02 --display 0 --write_video ../output_video.mp4 --write_video_with_audio

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libthe

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/output_video.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)